### Twilight Struggle

In [1]:
import numpy as np
import random

from twilight_map import *
from game_mechanics import *
from twilight_cards import *

### Game Instance

In [2]:
# this is a temporary measure. It should go into UI.new()
g = Game()
g.ui.run()

Initalising game..

----- USSR Player: -----
You may play a card. Type in the card index.
1	Asia_Scoring
2	Europe_Scoring
3	Middle_East_Scoring
4	Duck_and_Cover - 3
5	Five_Year_Plan - 3
6	The_China_Card - 4
7	Socialist_Governments - 3
8	Fidel - 2
9	Vietnam_Revolts - 2
10	Blockade - 1
11	Korean_War - 2
12	Romanian_Abdication - 1
13	Arab_Israeli_War - 2
14	COMECON - 3
15	Nasser - 1
16	Warsaw_Pact_Formed - 3
17	De_Gaulle_Leads_France - 3
18	Captured_Nazi_Scientist - 1
> q


AttributeError: 'NoneType' object has no attribute 'info'